# Setup

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoints

In [41]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [44]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [45]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [46]:
def draw_styled_landmarks(image, results):
    
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [ ]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# Extract Keypoint Values

In [48]:
len(results.face_landmarks.landmark)

468

In [49]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [50]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [51]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [52]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [53]:
result_test = extract_keypoints(results)

In [54]:
result_test

array([ 0.4995819 ,  0.55834734, -1.14279866, ...,  0.        ,
        0.        ,  0.        ])

In [ ]:
np.save('0', result_test)

In [ ]:
np.load('0.npy')

# Setup Folders for Collection

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join(r"R:\CV\Chrad\Data") 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [3]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect Keypoint Values for Training and Testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()
                #cv2.waitKey(5000)

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# Preprocess Data and Create Labels and Features

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
label_map = {label:num for num, label in enumerate(actions)}

In [6]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [7]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [8]:
np.array(sequences).shape

(90, 30, 1662)

In [9]:
np.array(labels).shape

(90,)

In [10]:
X = np.array(sequences)
X.shape

(90, 30, 1662)

In [11]:
y = to_categorical(labels).astype(int)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=45)

In [13]:
y_test.shape

(5, 3)

# Build and Train LSTM Neural Network

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [21]:
#tensorboard
log_dir = os.path.join('R://CV//Chrad', 'Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# checkpoint
filepath="Models/weights-improvement-{epoch:02d}-{categorical_accuracy:.2f}.ckpt"
checkpoint = ModelCheckpoint(filepath, monitor='categorical_accuracy', verbose=1,
                             save_weights_only=True, save_best_only=True, mode='max')

callbacks_list = [checkpoint, tb_callback]

In [22]:
os.listdir(log_dir)

['train']

In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
model.compile(optimizer='Nadam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=500, callbacks=callbacks_list)

Epoch 1/500
3/3 [==============================] - ETA: 0s - loss: 2.0644 - categorical_accuracy: 0.3412
Epoch 1: categorical_accuracy improved from -inf to 0.34118, saving model to Models/weights-improvement-01-0.34\action.hdf5
3/3 [==============================] - 10s 1s/step - loss: 2.0644 - categorical_accuracy: 0.3412
Epoch 2/500
3/3 [==============================] - ETA: 0s - loss: 9.9594 - categorical_accuracy: 0.3412 
Epoch 2: categorical_accuracy did not improve from 0.34118
3/3 [==============================] - 4s 1s/step - loss: 9.9594 - categorical_accuracy: 0.3412
Epoch 3/500
3/3 [==============================] - ETA: 0s - loss: 14.7595 - categorical_accuracy: 0.4706
Epoch 3: categorical_accuracy improved from 0.34118 to 0.47059, saving model to Models/weights-improvement-03-0.47\action.hdf5
3/3 [==============================] - 4s 1s/step - loss: 14.7595 - categorical_accuracy: 0.4706
Epoch 4/500
3/3 [==============================] - ETA: 0s - loss: 42.5995 - catego

3/3 [==============================] - 4s 2s/step - loss: 1.9516 - categorical_accuracy: 0.4471
Epoch 32/500
3/3 [==============================] - ETA: 0s - loss: 1.9676 - categorical_accuracy: 0.4000
Epoch 32: categorical_accuracy did not improve from 0.55294
3/3 [==============================] - 4s 1s/step - loss: 1.9676 - categorical_accuracy: 0.4000
Epoch 33/500
3/3 [==============================] - ETA: 0s - loss: 1.7470 - categorical_accuracy: 0.3176
Epoch 33: categorical_accuracy did not improve from 0.55294
3/3 [==============================] - 3s 1s/step - loss: 1.7470 - categorical_accuracy: 0.3176
Epoch 34/500
3/3 [==============================] - ETA: 0s - loss: 2.6398 - categorical_accuracy: 0.3059
Epoch 34: categorical_accuracy did not improve from 0.55294
3/3 [==============================] - 4s 1s/step - loss: 2.6398 - categorical_accuracy: 0.3059
Epoch 35/500
3/3 [==============================] - ETA: 0s - loss: 1.6691 - categorical_accuracy: 0.5059
Epoch 35: ca

3/3 [==============================] - 5s 2s/step - loss: 0.6128 - categorical_accuracy: 0.8118
Epoch 93/500
3/3 [==============================] - ETA: 0s - loss: 0.5870 - categorical_accuracy: 0.6706
Epoch 93: categorical_accuracy did not improve from 0.81176
3/3 [==============================] - 4s 1s/step - loss: 0.5870 - categorical_accuracy: 0.6706
Epoch 94/500
3/3 [==============================] - ETA: 0s - loss: 0.5590 - categorical_accuracy: 0.7176
Epoch 94: categorical_accuracy did not improve from 0.81176
3/3 [==============================] - 5s 2s/step - loss: 0.5590 - categorical_accuracy: 0.7176
Epoch 95/500
3/3 [==============================] - ETA: 0s - loss: 1.0436 - categorical_accuracy: 0.6235
Epoch 95: categorical_accuracy did not improve from 0.81176
3/3 [==============================] - 5s 2s/step - loss: 1.0436 - categorical_accuracy: 0.6235
Epoch 96/500
3/3 [==============================] - ETA: 0s - loss: 0.5332 - categorical_accuracy: 0.7412
Epoch 96: ca

3/3 [==============================] - 5s 2s/step - loss: 0.4991 - categorical_accuracy: 0.7176
Epoch 124/500
3/3 [==============================] - ETA: 0s - loss: 1.0793 - categorical_accuracy: 0.5176
Epoch 124: categorical_accuracy did not improve from 0.82353
3/3 [==============================] - 5s 1s/step - loss: 1.0793 - categorical_accuracy: 0.5176
Epoch 125/500
3/3 [==============================] - ETA: 0s - loss: 3.2770 - categorical_accuracy: 0.2941
Epoch 125: categorical_accuracy did not improve from 0.82353
3/3 [==============================] - 4s 2s/step - loss: 3.2770 - categorical_accuracy: 0.2941
Epoch 126/500
3/3 [==============================] - ETA: 0s - loss: 1.7726 - categorical_accuracy: 0.4824
Epoch 126: categorical_accuracy did not improve from 0.82353
3/3 [==============================] - 5s 2s/step - loss: 1.7726 - categorical_accuracy: 0.4824
Epoch 127/500
3/3 [==============================] - ETA: 0s - loss: 0.9320 - categorical_accuracy: 0.6000
Epoch

3/3 [==============================] - 5s 2s/step - loss: 0.4596 - categorical_accuracy: 0.7059
Epoch 155/500
3/3 [==============================] - ETA: 0s - loss: 0.3098 - categorical_accuracy: 0.8471
Epoch 155: categorical_accuracy did not improve from 0.94118
3/3 [==============================] - 4s 2s/step - loss: 0.3098 - categorical_accuracy: 0.8471
Epoch 156/500
3/3 [==============================] - ETA: 0s - loss: 0.3069 - categorical_accuracy: 0.9176
Epoch 156: categorical_accuracy did not improve from 0.94118
3/3 [==============================] - 5s 2s/step - loss: 0.3069 - categorical_accuracy: 0.9176
Epoch 157/500
3/3 [==============================] - ETA: 0s - loss: 0.4170 - categorical_accuracy: 0.7529
Epoch 157: categorical_accuracy did not improve from 0.94118
3/3 [==============================] - 5s 2s/step - loss: 0.4170 - categorical_accuracy: 0.7529
Epoch 158/500
3/3 [==============================] - ETA: 0s - loss: 0.6796 - categorical_accuracy: 0.6353
Epoch

3/3 [==============================] - 5s 2s/step - loss: 1.3510 - categorical_accuracy: 0.5412
Epoch 186/500
3/3 [==============================] - ETA: 0s - loss: 0.4135 - categorical_accuracy: 0.8235
Epoch 186: categorical_accuracy did not improve from 0.95294
3/3 [==============================] - 5s 2s/step - loss: 0.4135 - categorical_accuracy: 0.8235
Epoch 187/500
3/3 [==============================] - ETA: 0s - loss: 0.3321 - categorical_accuracy: 0.8235
Epoch 187: categorical_accuracy did not improve from 0.95294
3/3 [==============================] - 5s 2s/step - loss: 0.3321 - categorical_accuracy: 0.8235
Epoch 188/500
3/3 [==============================] - ETA: 0s - loss: 0.6387 - categorical_accuracy: 0.7294
Epoch 188: categorical_accuracy did not improve from 0.95294
3/3 [==============================] - 4s 1s/step - loss: 0.6387 - categorical_accuracy: 0.7294
Epoch 189/500
3/3 [==============================] - ETA: 0s - loss: 0.2189 - categorical_accuracy: 0.9529
Epoch

Epoch 217/500
3/3 [==============================] - ETA: 0s - loss: 0.4389 - categorical_accuracy: 0.8118
Epoch 217: categorical_accuracy did not improve from 0.95294
3/3 [==============================] - 4s 1s/step - loss: 0.4389 - categorical_accuracy: 0.8118
Epoch 218/500
3/3 [==============================] - ETA: 0s - loss: 0.6696 - categorical_accuracy: 0.6118
Epoch 218: categorical_accuracy did not improve from 0.95294
3/3 [==============================] - 4s 1s/step - loss: 0.6696 - categorical_accuracy: 0.6118
Epoch 219/500
3/3 [==============================] - ETA: 0s - loss: 0.9322 - categorical_accuracy: 0.6118
Epoch 219: categorical_accuracy did not improve from 0.95294
3/3 [==============================] - 4s 1s/step - loss: 0.9322 - categorical_accuracy: 0.6118
Epoch 220/500
3/3 [==============================] - ETA: 0s - loss: 0.4219 - categorical_accuracy: 0.6588
Epoch 220: categorical_accuracy did not improve from 0.95294
3/3 [==============================] - 4

KeyboardInterrupt: 

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

# Make Predictions

In [27]:
res = model.predict(X_test)

In [28]:
actions[np.argmax(res[4])]

'hello'

In [29]:
actions[np.argmax(y_test[4])]

'hello'

# Save Weights

In [30]:
#model.save('action.h5')
model.save('models/action')

INFO:tensorflow:Assets written to: models/action\assets


INFO:tensorflow:Assets written to: models/action\assets


In [ ]:
#del model

In [ ]:
# from keras import models

# #model.load_weights('action.h5')
# model = keras.models.load_model('models/action')

In [75]:
from keras.models import load_model

model = load_model('action.hdf5')

# Evaluation using Confusion Matrix and Accuracy

In [76]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [77]:
yhat = model.predict(X_test)

In [78]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [79]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[0, 1],
        [0, 4]],

       [[4, 0],
        [1, 0]]], dtype=int64)

In [80]:
accuracy_score(ytrue, yhat)

0.8

# Test in Real Time

In [81]:
from scipy import stats

In [82]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [83]:
plt.figure(figsize=(10,10))
plt.imshow(prob_viz(res, actions, image, colors))

AttributeError: module 'PIL.Image' has no attribute 'Resampling'

<Figure size 720x720 with 1 Axes>

In [84]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.soluti

iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou


In [57]:
cap.release()
cv2.destroyAllWindows()